In [1]:
import os
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Function
import torchvision
import time
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data as util_data
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import torchvision.datasets as dsets
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchvision import models
import cv2
import time

import pandas as pd
import matplotlib.pyplot as plt
import shutil
from typing import Any
import random
import torchvision

In [2]:
dataset_root = '/kaggle/input/palm-p600/palm_6000'
#dataset_root = 'F:\深度哈希\新建文件夹\data'

dataset_train_root = os.path.join(dataset_root, 'train')
dataset_val_root = os.path.join(dataset_root, 'val/')
dataset_all_dataset_root = os.path.join(dataset_root, 'all_dataset')


# win设为0 
num_workers = 8

num_threads =8 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
num_epochs =100
# epoch_lr_decrease = 100
learning_rate = 0.004
encode_length =256
num_classes =60
 

In [4]:
# 配置训练集DataSet类
class SessionDataset(Dataset):
    
  
    def __init__(self, root, transform=None):
   
        self.labels = np.array([int(x.split('_')[0]) for x in os.listdir(path=root)])

      

        self.image_files = np.array([x.path for x in os.scandir(path=root)])
        
     
        self.transform = transform
        
    
    def __getitem__(self, index):
        
    
        img = cv2.imread(self.image_files[index])
        
    
        label = self.labels[index]


        if self.transform:
            img = self.transform(img)
        
        return img, label
    
    
    def __len__(self):
        return np.size(self.image_files)
        

In [5]:
data_transform = {
    "train": transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop((224, 224)),
#         transforms.RandomResizedCrop((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),#转为Tensor
        transforms.Normalize((0.51568358, 0.51568358, 0.51568358), (0.10332626, 0.10332626, 0.10332626)),#标准化处理
        
    ]),
    "val": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.51568358, 0.51568358, 0.51568358), (0.10332626, 0.10332626, 0.10332626))

    ]),
    "all_dataset": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.51568358, 0.51568358, 0.51568358), (0.10332626, 0.10332626, 0.10332626))

    ]),
    
}

In [6]:
train_batch_size =32#64
val_batch_size = 32




session_train_dataset = SessionDataset(root=dataset_train_root,
                                    transform=data_transform['train'])

train_loader = DataLoader(session_train_dataset,
                         batch_size=train_batch_size,
                         shuffle=True,
                         num_workers=num_workers)


session_val_dataset = SessionDataset(root=dataset_val_root,
                                    transform=data_transform['val'])
val_loader = DataLoader(session_val_dataset,
                         batch_size=val_batch_size,
                         shuffle=False,
                         num_workers=num_workers)

# 包含整个验证集

sessiondatabaseDataset = SessionDataset(root=dataset_all_dataset_root,
                                    transform=data_transform['all_dataset'])

database_loader = DataLoader(sessiondatabaseDataset,
                         batch_size=train_batch_size,
                         shuffle=False,
                         num_workers=num_workers)

train_num = len(session_train_dataset)#4800
val_num = len(session_val_dataset)#1200
datasetloader_num = len(sessiondatabaseDataset)#6000
print(train_num, val_num,datasetloader_num)


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


480 120 600


**Original resnet 18,50,100******

In [7]:
import torch
from torch import nn

__all__ = ['iresnet18', 'iresnet34', 'iresnet50', 'iresnet100', 'iresnet200']


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes,
                     out_planes,
                     kernel_size=3,
                     stride=stride,
                     padding=dilation,
                     groups=groups,
                     bias=False,
                     dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes,
                     out_planes,
                     kernel_size=1,
                     stride=stride,
                     bias=False)


class IBasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None,
                 groups=1, base_width=64, dilation=1):
        super(IBasicBlock, self).__init__()
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        self.bn1 = nn.BatchNorm2d(inplanes, eps=1e-05,)
        self.conv1 = conv3x3(inplanes, planes)
        self.bn2 = nn.BatchNorm2d(planes, eps=1e-05,)
        self.prelu = nn.PReLU(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn3 = nn.BatchNorm2d(planes, eps=1e-05,)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        out = self.bn1(x)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.prelu(out)
        out = self.conv2(out)
        out = self.bn3(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return out


class IResNet(nn.Module):
    fc_scale = 7 * 7
    def __init__(self,
                 block, layers, dropout=0, num_features=512, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None, fp16=False):
        super(IResNet, self).__init__()
        self.fp16 = fp16
        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes, eps=1e-05)
        self.prelu = nn.PReLU(self.inplanes)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=2)
        self.layer2 = self._make_layer(block,
                                       128,
                                       layers[1],
                                       stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block,
                                       256,
                                       layers[2],
                                       stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block,
                                       512,
                                       layers[3],
                                       stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.bn2 = nn.BatchNorm2d(512 * block.expansion, eps=1e-05,)
        self.dropout = nn.Dropout(p=dropout, inplace=True)
        self.fc = nn.Linear(4 * 512 * block.expansion * self.fc_scale, num_features)
        self.features = nn.BatchNorm1d(num_features, eps=1e-05)
        nn.init.constant_(self.features.weight, 1.0)
        self.features.weight.requires_grad = False

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, 0, 0.1)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, IBasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion, eps=1e-05, ),
            )
        layers = []
        layers.append(
            block(self.inplanes, planes, stride, downsample, self.groups,
                  self.base_width, previous_dilation))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(self.inplanes,
                      planes,
                      groups=self.groups,
                      base_width=self.base_width,
                      dilation=self.dilation))

        return nn.Sequential(*layers)

    def forward(self, x):
        with torch.cuda.amp.autocast(self.fp16):
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.prelu(x)
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)
            x = self.bn2(x)
            x = torch.flatten(x, 1)
            x = self.dropout(x)
        x = self.fc(x.float() if self.fp16 else x)
        x = self.features(x)
        return x


def _iresnet(arch, block, layers, pretrained, progress, **kwargs):
    model = IResNet(block, layers, **kwargs)
    if pretrained:
        raise ValueError()
    return model


def iresnet18(pretrained=False, progress=True, **kwargs):
    return _iresnet('iresnet18', IBasicBlock, [2, 2, 2, 2], pretrained,
                    progress, **kwargs)


 


added CBAM with resnet 50

In [8]:
iresnet18()

IResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (prelu): PReLU(num_parameters=64)
  (layer1): Sequential(
    (0): IBasicBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=64)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): IBas

Hashing Layer

In [9]:
import numpy as np
 

def logistic(x, k=1, x0=0):
    """
    Logistic function: L / (1 + exp(-k * (x - x0)))
    
    Parameters:
    - x: Input values
    - k: Steepness of the curve (default: 1)
    - x0: x-value of the sigmoid's midpoint (default: 0)
    
    Returns:
    - Result of the logistic function
    """
    return 1 / (1 + np.exp(-k * (x - x0)))


def new_dropout(x, level=0.5):  
    if level < 0. or level >= 1:#level是概率值，必须在0~1之间  
        raise Exception('Dropout level must be in interval [0, 1].')  
    retain_prob = 1. - level  
    #我们通过binomial函数，生成与x一样的维数向量。binomial函数就像抛硬币一样，我们可以把每个神经元当做抛硬币一样  
    #硬币 正面的概率为p，n表示每个神经元试验的次数  
    #因为我们每个神经元只需要抛一次就可以了所以n=1，size参数是我们有多少个硬币。 
     
   # sample=nn.random.binomial(n=1,p=retain_prob,size=x.shape)#即将生成一个0、1分布的向量，0表示这个神经元被屏蔽，不工作了，也就是dropout了  
    sample = torch.from_numpy(np.array(logistic(0.3, 100))).to(device)

#
  #  sample=np.array(logistic(0.36,x.dim()))#
#     print (sample)  
#     print(type(sample))
    
    x *=sample          #屏蔽某些神经元
    
    x /= retain_prob   #此处是dropout
    return x



In [10]:
class hash(Function):
    # 静态方法
    @staticmethod
    def forward(ctx, input):
        # ctx.save_for_backward(input)
        return torch.sign(input)

    @staticmethod
    def backward(ctx, grad_output):
        
        return grad_output

def hash_layer(input):
    return hash.apply(input)
class GreedyHash(nn.Module):
    def __init__(self):      
        super(GreedyHash, self).__init__()      
        self.wres = iresnet18(pretrained=False,num_features=num_classes).to(device)
        self.fc_plus = nn.Linear(num_classes, encode_length)
        self.fc = nn.Linear(encode_length, num_classes, bias=False)
    def get_hash_length(self):
        return self.encode_length
    def forward(self, x):
        x = self.wres(x)
        x = new_dropout(x)
        x = self.fc_plus(x)       
        code = hash_layer(x)      
        output = self.fc(code)        
        return output, x, code


In [11]:

net = GreedyHash().to(device)

In [12]:
class ArcFace(nn.Module):
    def __init__(self, s=64.0, m=0.5):
        super(ArcFace, self).__init__()
        self.s = s
        self.m = m

    def forward(self, cosine: torch.Tensor, label):
        index = torch.where(label != -1)[0]
        m_hot = torch.zeros(index.size()[0], cosine.size()[1], device=cosine.device)
        m_hot.scatter_(1, label[index, None], self.m)
        cosine.acos_()
        cosine[index] += m_hot
        cosine.cos_().mul_(self.s)
        return cosine


In [13]:

class FocalLoss(nn.Module):

    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = torch.nn.CrossEntropyLoss()

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

In [14]:

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)




In [15]:
def compress(train, test, model, classes=100):
    retrievalB = list([])
    retrievalL = list([])
    with torch.no_grad():
        for batch_step, (data, target) in enumerate(train):         
            var_data = data.to(device)            
            _,_, code = model(var_data)        
            retrievalB.extend(code.cpu().data.numpy())
            retrievalL.extend(target)
         
        queryB=list([])
        queryL=list([])
        for batch_step, (data, target) in enumerate(test):     
            var_data = data.to(device)
            _,_, code = model(var_data)
            
            queryB.extend(code.cpu().data.numpy())
            queryL.extend(target)
            
        retrievalB=np.array(retrievalB)
        retrievalL=np.eye(classes)[np.array(retrievalL)]
            
        queryB=np.array(queryB)
        queryL=np.eye(classes)[np.array(queryL)]   
        
            
        return retrievalB, retrievalL, queryB, queryL


    
    
def calculate_hamming(B1, B2):
    """
    :param B1:  vector [n]
    :param B2:  vector [r*n]
    :return: hamming distance [r]
    """
    q = B2.shape[1] # max inner product value
    distH = 0.5 * (q - np.dot(B1, B2.transpose()))
    return distH


def calculate_map(qB, rB, queryL, retrievalL):
    """
       :param qB: {-1,+1}^{mxq} query bits
       :param rB: {-1,+1}^{nxq} retrieval bits
       :param queryL: {0,1}^{mxl} query label
       :param retrievalL: {0,1}^{nxl} retrieval label
       :return:
    """
    num_query = queryL.shape[0]
    map = 0
    for iter in range(num_query):
       
        gnd = (np.dot(queryL[iter, :], retrievalL.transpose()) > 0).astype(np.float32)
        
        tsum = np.sum(gnd).astype(int)
        if tsum == 0:
            continue
        
        hamm = calculate_hamming(qB[iter, :], rB)
        ind = np.argsort(hamm)
        gnd = gnd[ind]

        count = np.linspace(1, tsum, tsum) # [1,2, tsum]
        tindex = np.asarray(np.where(gnd == 1)) + 1.0
        map_ = np.mean(count / (tindex))
       
        map = map + map_
    map = map / num_query
    return map



def calculate_top_map(qB, rB, queryL, retrievalL, topk):
    """
    :param qB: {-1,+1}^{mxq} query bits
    :param rB: {-1,+1}^{nxq} retrieval bits
    :param queryL: {0,1}^{mxl} query label
    :param retrievalL: {0,1}^{nxl} retrieval label
    :param topk:
    :return:
    """
    num_query = queryL.shape[0]
    topkmap = 0
    for iter in range(num_query):
        
       
        gnd = (np.dot(queryL[iter, :], retrievalL.transpose()) > 0).astype(np.float32)
        hamm = calculate_hamming(qB[iter, :], rB)
        ind = np.argsort(hamm)
        gnd = gnd[ind]

        tgnd = gnd[0:topk]
        tsum = np.sum(tgnd).astype(int)
        if tsum == 0:
            continue
        count = np.linspace(1, tsum, tsum)

        tindex = np.asarray(np.where(tgnd == 1)) + 1.0
        topkmap_ = np.mean(count / (tindex))
      
        topkmap = topkmap + topkmap_
    topkmap = topkmap / num_query
    return topkmap



def myCalcTopMap(rB, qB, retrievalL, queryL, topk):
    
    
    num_query = queryL.shape[0]
    
    topkmap = 0
    
    
    for iter in tqdm(range(num_query)):
        
        
        gnd = (np.dot(queryL[iter, :], retrievalL.transpose()) > 0).astype(np.float32)
        
        
        hamm = CalcHammingDist(qB[iter, :], rB)
        
        
        ind = np.argsort(hamm)
        
       
        gnd = gnd[ind]
        
        
        tgnd = gnd[0:topk]
        
       
        tsum = np.sum(tgnd).astype(int)
        if tsum == 0:
            continue
        
       
        count = np.linspace(1, tsum, tsum)
        
        
        tindex = np.asarray(np.where(tgnd == 1)) + 1.0
        
        
        topkmap_ = np.mean(count / (tindex))
        
        topkmap = topkmap + topkmap_
   
    topkmap = topkmap / num_query
    
    return topkmap

In [16]:
#TEST
# resnet

In [17]:
# net

In [ ]:
# 训练过程

best = 0.0

torch.set_num_threads(num_threads)

if torch.cuda.device_count() > 1:
    net = nn.DataParallel(net)    
net.to(device)
# Train the Model
for epoch in range(num_epochs):  
    net.train()

    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs, feature, _ = net(images)


        loss1 = criterion(outputs, labels)
  
        loss2 = torch.mean(torch.abs(torch.pow(torch.abs(feature) - torch.ones(feature.size()).to(device), 3)))
        loss = loss1 + 0.1 * loss2
        loss.backward()
        optimizer.step()

#         if (i + 1) % (len(oxford102TrainDataset) // batch_size / 2) == 0:
        print ('Epoch [%d/%d], Iter [%d/%d] Loss1: %.4f Loss2: %.4f'
               % (epoch + 1, num_epochs, i + 1, len(sessiondatabaseDataset) // train_batch_size,
                  loss1.item(), loss2.item()))

   
    net.eval()  
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
           
            outputs, _, _ = net(images)
          
            _, predicted = torch.max(outputs.cpu().data, 1)

            total += labels.size(0)

            correct += (predicted == labels).sum()

    print('Test Accuracy of the model: %.2f %%' % (100.0 * correct / total))

    if 1.0 * correct / total > best:
        best = 1.0 * correct / total
        torch.save(net.state_dict(), 'temp44.pkl')    
    print('best: %.2f %%' % (best * 100.0))
    net.eval()
    retrievalB, retrievalL, queryB, queryL = compress(train_loader, val_loader, net)
    print('---calculate map1---')
    result1 = calculate_map(qB=queryB, rB=retrievalB, queryL=queryL, retrievalL=retrievalL)
    #torch.save(net.state_dict(), 'map1_{}_module_all_obj.pkl'.format(round(result1,3)))
    print(result1)
    retrievalB, retrievalL, queryB, queryL = compress(database_loader, val_loader, net)
    print('---calculate map2---')
    result2 = calculate_map(qB=queryB, rB=retrievalB, queryL=queryL, retrievalL=retrievalL)
    print(result2)
    

Epoch [1/100], Iter [1/18] Loss1: 4.6536 Loss2: 0.3364
Epoch [1/100], Iter [2/18] Loss1: 4.8666 Loss2: 0.4167
Epoch [1/100], Iter [3/18] Loss1: 4.5895 Loss2: 0.3801
Epoch [1/100], Iter [4/18] Loss1: 4.3597 Loss2: 0.4041
Epoch [1/100], Iter [5/18] Loss1: 4.0144 Loss2: 0.5041
Epoch [1/100], Iter [6/18] Loss1: 4.4218 Loss2: 0.4224
Epoch [1/100], Iter [7/18] Loss1: 4.5003 Loss2: 0.4443
Epoch [1/100], Iter [8/18] Loss1: 4.3988 Loss2: 0.4843
Epoch [1/100], Iter [9/18] Loss1: 4.2672 Loss2: 0.3736
Epoch [1/100], Iter [10/18] Loss1: 4.1235 Loss2: 0.4155
Epoch [1/100], Iter [11/18] Loss1: 4.2033 Loss2: 0.3858
Epoch [1/100], Iter [12/18] Loss1: 4.1936 Loss2: 0.3505
Epoch [1/100], Iter [13/18] Loss1: 4.2194 Loss2: 0.4665
Epoch [1/100], Iter [14/18] Loss1: 4.0284 Loss2: 0.3838
Epoch [1/100], Iter [15/18] Loss1: 3.8886 Loss2: 0.3464
Test Accuracy of the model: 3.33 %
best: 3.33 %
---calculate map1---
0.1174570714922674
---calculate map2---
0.4147603402755616
Epoch [2/100], Iter [1/18] Loss1: 4.0468 